In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('reconstructed_data.csv') # load the data of EMG signals

df.set_index('sequence',inplace=True) # set the index to the sequence column

X = df.drop('label',axis=1) # drop the label column from the data
Y = df[['label']] # get the label column to be used as the target
#Y = df.drop(df.columns[df.apply(lambda x: x.name != 'label')],axis=1)

In [3]:
X

,0,1,2,3,4,5,6,7,8,9,...,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
sequence,,,,,,,,,,,,,,,,,,,,,
0,0.100343,0.100350,0.100370,0.100400,0.100435,0.100473,0.100511,0.100550,0.100588,0.100627,...,0.099908,0.099945,0.099982,0.100019,0.100056,0.100093,0.100129,0.100161,0.100185,0.100198
0,0.110339,0.110336,0.110327,0.110314,0.110298,0.110281,0.110264,0.110247,0.110230,0.110213,...,0.110540,0.110523,0.110505,0.110488,0.110471,0.110454,0.110437,0.110423,0.110412,0.110405
0,0.036453,0.036453,0.036452,0.036451,0.036449,0.036447,0.036446,0.036444,0.036442,0.036440,...,0.036475,0.036473,0.036471,0.036469,0.036467,0.036466,0.036464,0.036462,0.036461,0.036460
0,0.020544,0.020545,0.020549,0.020556,0.020563,0.020571,0.020580,0.020588,0.020596,0.020604,...,0.020446,0.020455,0.020463,0.020472,0.020480,0.020488,0.020496,0.020503,0.020509,0.020512
1,0.103840,0.103843,0.103849,0.103859,0.103871,0.103883,0.103896,0.103909,0.103922,0.103935,...,0.103698,0.103710,0.103722,0.103734,0.103746,0.103758,0.103770,0.103780,0.103788,0.103792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,0.014313,0.014314,0.014318,0.014323,0.014329,0.014336,0.014343,0.014350,0.014357,0.014364,...,0.014234,0.014241,0.014248,0.014254,0.014261,0.014268,0.014274,0.014280,0.014285,0.014287
1999,0.093110,0.093094,0.093051,0.092987,0.092911,0.092829,0.092746,0.092663,0.092580,0.092497,...,0.094083,0.093999,0.093915,0.093831,0.093747,0.093663,0.093584,0.093513,0.093459,0.093430
1999,0.118611,0.118603,0.118581,0.118549,0.118510,0.118469,0.118427,0.118385,0.118343,0.118301,...,0.119101,0.119059,0.119016,0.118974,0.118932,0.118890,0.118850,0.118814,0.118787,0.118772


In [4]:
Y

,label
sequence,
0,0
0,0
0,0
0,0
1,0
...,...
1998,9
1999,9
1999,9


In [5]:
Xarray = X.to_numpy() # convert the data to numpy array
Xarray = Xarray.reshape(2000,4,-1) # reshape the data to 3D numpy array
Xarray

array([[[0.10034339, 0.10035037, 0.10037013, ..., 0.10016064,
         0.10018479, 0.10019794],
        [0.11033939, 0.11033625, 0.11032735, ..., 0.11042262,
         0.11041153, 0.1104055 ],
        [0.03645344, 0.03645311, 0.03645218, ..., 0.03646228,
         0.03646109, 0.03646044],
        [0.02054354, 0.02054505, 0.02054933, ..., 0.02050337,
         0.02050873, 0.02051165]],

       [[0.10384047, 0.10384279, 0.10384935, ..., 0.10378019,
         0.10378812, 0.10379244],
        [0.10463462, 0.10463483, 0.10463543, ..., 0.10462849,
         0.10462936, 0.10462983],
        [0.03283462, 0.03283406, 0.03283249, ..., 0.03285   ,
         0.03284789, 0.03284675],
        [0.01517424, 0.01517493, 0.0151769 , ..., 0.01515572,
         0.0151582 , 0.01515955]],

       [[0.1147981 , 0.11480091, 0.11480885, ..., 0.11472528,
         0.11473484, 0.11474005],
        [0.10165947, 0.10165994, 0.10166129, ..., 0.10164754,
         0.10164906, 0.1016499 ],
        [0.0358137 , 0.03581332, 0.0

In [6]:
Yarray = Y.to_numpy() # convert the target to numpy array
Yarray = Yarray.reshape(2000,4,-1) # reshape the target to 3D numpy array
Yarray = np.mean(Yarray,axis=1).astype(int) # get the mean of the target and convert it to integer to make it a 2D numpy array

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(Xarray, Yarray, test_size = 0.3, random_state=3) # split the data into training and testing set

In [8]:
Y_train.shape, Y_test.shape # check the shape of the target

((1400, 1), (600, 1))

In [9]:
from sktime.transformations.panel.rocket import Rocket
rocket = Rocket(num_kernels=1000,n_jobs=-1) # create a rocket object. set the number of kernels to 1000(default is 10000) and use all available cores

In [10]:
X_train = from_3d_numpy_to_nested(X_train) # convert the training data to nested format
rocket.fit(X_train, Y_train) # fit the rocket object to the training data
X_train = rocket.transform(X_train) # transform the training data

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [11]:
X_test = from_3d_numpy_to_nested(X_test) # convert the testing data to nested format
X_test = rocket.transform(X_test) # transform the testing data

In [12]:
X_train.shape, X_test.shape

((1400, 2000), (600, 2000))

In [13]:
from sklearn.linear_model import RidgeClassifierCV
clf1 = RidgeClassifierCV() # create a RidgeClassifier object
clf1.fit(X_train, Y_train) # fit the classifier with the training data

Y_pred = clf1.predict(X_test) # predict the target of the testing data
print(classification_report(Y_pred,Y_test)) # print the classification report to check the performance of the model

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:1190: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.84      0.88      0.86        67
           1       0.95      0.85      0.90        62
           2       0.96      0.96      0.96        57
           3       0.75      0.83      0.79        58
           4       0.91      0.81      0.86        75
           5       0.90      0.93      0.91        56
           6       0.85      0.93      0.89        55
           7       0.72      0.79      0.75        61
           8       0.68      0.77      0.72        47
           9       0.94      0.73      0.82        62

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.85      0.85      0.85       600



In [14]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=100) # create a RandomForestClassifier object
clf2.fit(X_train,Y_train) # fit the classifier with the training data

Y_pred = clf2.predict(X_test) # predict the target of the testing data
print(classification_report(Y_pred,Y_test)) # print the classification report to check the performance of the model

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.96      0.86      0.91        78
           1       0.98      0.87      0.92        63
           2       0.81      0.88      0.84        52
           3       0.69      0.77      0.73        57
           4       0.84      0.79      0.81        71
           5       0.91      0.91      0.91        58
           6       0.72      0.90      0.80        48
           7       0.76      0.85      0.80        60
           8       0.74      0.80      0.76        49
           9       0.96      0.72      0.82        64

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.85      0.83      0.84       600



In [15]:
from sklearn.svm import SVC
clf3 = SVC(kernel='linear')
clf3.fit(X_train,Y_train)

Y_pred = clf3.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.83      0.78      0.81        74
           1       0.86      0.87      0.86        55
           2       0.89      0.98      0.94        52
           3       0.72      0.71      0.71        65
           4       0.88      0.79      0.83        75
           5       0.93      0.95      0.94        57
           6       0.82      0.80      0.81        61
           7       0.66      0.71      0.68        62
           8       0.60      0.74      0.67        43
           9       0.77      0.66      0.71        56

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600



In [16]:
from sklearn.ensemble import GradientBoostingClassifier
clf4 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf4.fit(X_train,Y_train)

Y_pred = clf4.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.73      0.89      0.80        57
           1       0.82      0.81      0.81        57
           2       0.75      0.80      0.77        54
           3       0.58      0.54      0.56        69
           4       0.81      0.70      0.75        77
           5       0.76      0.81      0.79        54
           6       0.63      0.79      0.70        48
           7       0.55      0.59      0.57        63
           8       0.58      0.53      0.56        58
           9       0.75      0.57      0.65        63

    accuracy                           0.69       600
   macro avg       0.70      0.70      0.70       600
weighted avg       0.70      0.69      0.69       600



In [17]:
from xgboost import XGBClassifier
clf5 = XGBClassifier()
clf5.fit(X_train,Y_train)

Y_pred = clf5.predict(X_test)
print(classification_report(Y_pred,Y_test))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91        78
           1       0.98      0.90      0.94        61
           2       0.91      0.98      0.95        53
           3       0.83      0.84      0.83        63
           4       0.88      0.87      0.87        68
           5       0.93      0.92      0.92        59
           6       0.73      0.90      0.81        49
           7       0.75      0.81      0.78        62
           8       0.75      0.83      0.79        48
           9       0.90      0.73      0.80        59

    accuracy                           0.86       600
   macro avg       0.86      0.86      0.86       600
weighted avg       0.87      0.86      0.86       600



In [18]:
from lightgbm import LGBMClassifier
clf6 = LGBMClassifier()
clf6.fit(X_train,Y_train)

Y_pred = clf6.predict(X_test)

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 433099
[LightGBM] [Info] Number of data points in the train set: 1400, number of used features: 1879
[LightGBM] [Info] Start training from score -2.376693
[LightGBM] [Info] Start training from score -2.274414
[LightGBM] [Info] Start training from score -2.281383
[LightGBM] [Info] Start training from score -2.331573
[LightGBM] [Info] Start training from score -2.353878
[LightGBM] [Info] Start training from score -2.288400
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.353878
[LightGBM] [Info] Start training from score -2.253795
[LightGBM] [Info] Start training from score -2.220347
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

In [19]:
print(classification_report(Y_pred,Y_test))

              precision    recall  f1-score   support

           0       0.96      0.83      0.89        81
           1       0.98      0.86      0.92        64
           2       0.93      0.95      0.94        56
           3       0.75      0.87      0.81        55
           4       0.88      0.81      0.84        73
           5       0.93      0.93      0.93        58
           6       0.77      0.92      0.84        50
           7       0.70      0.82      0.76        57
           8       0.74      0.81      0.77        48
           9       0.92      0.76      0.83        58

    accuracy                           0.85       600
   macro avg       0.86      0.86      0.85       600
weighted avg       0.86      0.85      0.86       600



In [20]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf7 = QuadraticDiscriminantAnalysis()
clf7.fit(X_train,Y_train)

Y_pred = clf7.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


              precision    recall  f1-score   support

           0       0.10      0.18      0.13        38
           1       0.32      0.26      0.29        70
           2       0.19      0.15      0.17        75
           3       0.16      0.13      0.14        79
           4       0.27      0.21      0.24        84
           5       0.22      0.17      0.20        75
           6       0.13      0.24      0.17        34
           7       0.13      0.19      0.16        47
           8       0.09      0.10      0.10        51
           9       0.17      0.17      0.17        47

    accuracy                           0.18       600
   macro avg       0.18      0.18      0.17       600
weighted avg       0.19      0.18      0.18       600



In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf8 = LinearDiscriminantAnalysis()
clf8.fit(X_train,Y_train)

Y_pred = clf8.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.43      0.16      0.23       188
           1       0.20      0.33      0.25        33
           2       0.46      0.14      0.22       181
           3       0.06      0.12      0.08        34
           4       0.00      0.00      0.00         2
           5       0.26      0.16      0.20        91
           6       0.02      0.25      0.03         4
           7       0.01      0.25      0.03         4
           8       0.11      0.20      0.14        30
           9       0.10      0.15      0.12        33

    accuracy                           0.17       600
   macro avg       0.17      0.18      0.13       600
weighted avg       0.34      0.17      0.20       600



In [22]:
from sklearn.neural_network import MLPClassifier
clf9 = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=1000)
clf9.fit(X_train,Y_train)

Y_pred = clf9.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.87      0.87      0.87        70
           1       0.89      0.94      0.92        53
           2       0.88      1.00      0.93        50
           3       0.66      0.84      0.74        50
           4       0.93      0.70      0.79        89
           5       0.97      0.88      0.92        64
           6       0.83      0.79      0.81        63
           7       0.81      0.72      0.76        75
           8       0.60      0.86      0.71        37
           9       0.77      0.76      0.76        49

    accuracy                           0.82       600
   macro avg       0.82      0.84      0.82       600
weighted avg       0.84      0.82      0.82       600



In [23]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf10 = PassiveAggressiveClassifier()
clf10.fit(X_train,Y_train)

Y_pred = clf10.predict(X_test)
print(classification_report(Y_pred,Y_test))

/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.79      0.82      0.80        67
           1       0.93      0.83      0.87        63
           2       0.84      0.98      0.91        49
           3       0.73      0.77      0.75        61
           4       0.87      0.83      0.85        70
           5       0.88      0.93      0.90        55
           6       0.90      0.76      0.82        71
           7       0.72      0.73      0.72        66
           8       0.58      0.86      0.70        36
           9       0.85      0.66      0.75        62

    accuracy                           0.81       600
   macro avg       0.81      0.82      0.81       600
weighted avg       0.82      0.81      0.81       600



In [24]:
from catboost import CatBoostClassifier
clf11 = CatBoostClassifier()
clf11.fit(X_train,Y_train)

Y_pred = clf11.predict(X_test)

Learning rate set to 0.080496
0:	learn: 2.1856477	total: 958ms	remaining: 15m 57s
1:	learn: 2.0874914	total: 1.75s	remaining: 14m 31s
2:	learn: 1.9910772	total: 2.45s	remaining: 13m 35s
3:	learn: 1.9128123	total: 3.29s	remaining: 13m 38s
4:	learn: 1.8376708	total: 4.13s	remaining: 13m 42s
5:	learn: 1.7646793	total: 4.85s	remaining: 13m 23s
6:	learn: 1.6970290	total: 5.67s	remaining: 13m 24s
7:	learn: 1.6327707	total: 6.4s	remaining: 13m 13s
8:	learn: 1.5631452	total: 7.12s	remaining: 13m 3s
9:	learn: 1.5092648	total: 7.88s	remaining: 13m
10:	learn: 1.4563799	total: 8.66s	remaining: 12m 58s
11:	learn: 1.4113627	total: 9.39s	remaining: 12m 53s
12:	learn: 1.3665723	total: 10.1s	remaining: 12m 49s
13:	learn: 1.3269601	total: 10.8s	remaining: 12m 42s
14:	learn: 1.2904288	total: 11.5s	remaining: 12m 37s
15:	learn: 1.2575229	total: 12.3s	remaining: 12m 34s
16:	learn: 1.2260619	total: 13s	remaining: 12m 30s
17:	learn: 1.1939209	total: 13.7s	remaining: 12m 26s
18:	learn: 1.1610965	total: 14.4s	

In [25]:
print(classification_report(Y_pred,Y_test))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90        77
           1       0.96      0.93      0.95        58
           2       0.93      1.00      0.96        53
           3       0.77      0.82      0.79        60
           4       0.90      0.79      0.84        76
           5       0.97      0.93      0.95        60
           6       0.78      0.96      0.86        49
           7       0.72      0.86      0.78        56
           8       0.75      0.82      0.78        49
           9       0.96      0.74      0.84        62

    accuracy                           0.86       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.86      0.87       600



XGBClassifier, LGBMClassifier, and CatBoostClassifier show good performance.
These are based on the gradient boosting algorithm.